In [1]:
import pandas as pd
import time
import dateparser
import pytz
import argparse
from datetime import datetime
from binance.client import Client
import os
from binance.exceptions import BinanceAPIException

In [2]:
api_key = "iZHsmlsCReb9S6zVO05Vxy8ONQYK8J3CfshgNiRh3HlRShPULMj8EYBClftHBqi1"
api_secret = "4IHk54oeSmmoXGQqWNgi24SJ1uHaTSEBfN48nOhYex8ATFFOj2WoWZQfDFD0pzu1"

In [3]:
client = Client(api_key, api_secret)
tz = pytz.timezone('UTC')
buy = {}
loss = []
quantity = {}

In [4]:
def date_to_milliseconds(date_str):
    # date_str format: "January 01, 2018", "11 hours ago UTC", "now UTC"
    epoch = datetime.utcfromtimestamp(0).replace(tzinfo=pytz.utc)
    d = dateparser.parse(date_str)
    if d.tzinfo is None or d.tzinfo.utcoffset(d) is None:
        d = d.replace(tzinfo=pytz.utc)
    return int((d - epoch).total_seconds() * 1000.0)

In [5]:
def interval_to_milliseconds(interval):
    # interval format: 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 1d, 3d, 1w
    ms = None
    seconds_per_unit = {
        "m": 60,
        "h": 60 * 60,
        "d": 24 * 60 * 60,
        "w": 7 * 24 * 60 * 60
    }
    unit = interval[-1]
    if unit in seconds_per_unit:
        try:
            ms = int(interval[:-1]) * seconds_per_unit[unit] * 1000
        except ValueError:
            pass
    return ms

In [6]:
def get_historical_klines(symbol, interval, start_str, end_str=None):
    """Get Historical Klines from Binance
    If using offset strings for dates add "UTC" to date string e.g. "now UTC", "11 hours ago UTC", "1 Dec, 2017"
    :param symbol: Name of symbol pair e.g BNBBTC
    :param interval: Biannce Kline interval
    :param start_str: Start date string in UTC format
    :param end_str: optional - end date string in UTC format
    :return: list of Open High Low Close Volume values
    """
    output_data = []
    limit = 50
    timeframe = interval_to_milliseconds(interval)
    start_ts = date_to_milliseconds(start_str)
    end_ts = None
    if end_str:
        end_ts = date_to_milliseconds(end_str)

    idx = 0
    # it can be difficult to know when a symbol was listed on Binance so allow start time to be before list date
    symbol_existed = False
    while True:
        temp_data = client.get_klines(
            symbol=symbol,
            interval=interval,
            limit=limit,
            startTime=start_ts,
            endTime=end_ts
        )
        # handle the case where our start date is before the symbol pair listed on Binance
        if not symbol_existed and len(temp_data):
            symbol_existed = True
        if symbol_existed:
            output_data += temp_data
            start_ts = temp_data[len(temp_data) - 1][0] + timeframe
        else:
            start_ts += timeframe
        idx += 1
        if len(temp_data) < limit:
            break
        # sleep after every 3rd call to be kind to the API
        if idx % 3 == 0:
            time.sleep(1)

    return output_data


In [7]:
def get_historic_klines(symbol, start, end, interval):
    klines = get_historical_klines(symbol, interval, start, end)
    #print("Klines")
    #print(klines[-1])
    ochl = []
    for kline in klines:
        time1 = int(kline[0])
        open1 = float(kline[1])
        high = float(kline[2])
        low = float(kline[3])
        close = float(kline[4])
        volume = float(kline[5])
        ochl.append([time1, open1, close, high, low, volume])
    #df = pd.DataFrame(ochl)
    #print(symbol)
    #print(df)
    #print(ochl[-1])
    '''
    fig, ax = plt.subplots()
    mpl_finance.candlestick_ochl(ax, ochl, width=1)
    ax.set(xlabel='Date', ylabel='Price', title='{} {}-{}'.format(symbol, start, end))
    plt.show(block=False)
    plt.pause(3)
    plt.close()
    '''
    return ochl


In [12]:
def Main():
    list_of_symbols = ['XMRUSDT', 'LINKUSDT', 'LTCUSDT', 'BNBUSDT', 'EOSUSDT', 'BTCUSDT', 'ETHUSDT', 'XRPUSDT']  # Symbols to be traded
    quantity_1 = 5 # any value between 1-4 : 1 =100%, 2=50%, 3 = 33%, 4 = 25%, 5 = 20% and so on...
    max_amount = 10000  # Maximum authorized amount
    loss_limit = -50  # Maximum loss limit to terminate the trading in dollar
    buy_percent = 0.004 # percent at which it should buy, currently 0.15% = 0.15/100 = 0.0015
    sell_percent = -0.006 # percent at which it should sell, currently 0.1%
    loss_percent = -0.002 # stop loss if price falls, currently -0.3%
    transaction = 150 # number of maximum transactions
    buy_range = 0.005  # allowed buy upto, currently 1.5%
    sleep_time = 20   # according to candle interval 15 for 5 MINUTE, 30 for 30 MINUTE, 45 for 1 HOUR
    spent_amount = 0
    count = 0
    buy_open = [] # to avoid multiple buy at same candle 
    high ={} 
    symbols_csv_files = []
    while True:
        client.get_deposit_address(asset='USDT')  # USDT or BTC

        try:
            for symbol in list_of_symbols:
                
                # My modifications
                ochl = get_historic_klines(symbol, "5 days ago UTC", "now UTC", Client.KLINE_INTERVAL_5MINUTE)
                open1, close, high1,high_past = ochl[-1][1], ochl[-1][2],ochl[-1][3],ochl[-2][3]
                
                if symbol not in symbols_csv_files:
                    df = pd.DataFrame(ochl, columns=["date", "open", "close", "high", "low", "volume"])
                    
                    f_name = str(symbol) + ".csv"
                    for i in df.index:
                        df.at[i,'date'] = datetime.fromtimestamp((int(df.at[i,'date']))/1000.0).date()
                    symbols_csv_files.append(symbol)
                    print(symbol)
                    print(df)
                    df.to_csv(f_name, columns=["date", "open", "close", "high", "low", "volume"])
                
                if symbol in high:
                    #print(high1)
                    if high1>high_past and open1 not in buy_open:
                        high[symbol] = high1
                    elif high1>high[symbol]:
                        high[symbol]=high1
                symbol = str(symbol)
                if open1 not in buy_open:        
                    if (close  >= (1+buy_percent) * open1) and (symbol not in buy.keys()) and close < (1+buy_range) * open1:
                        if spent_amount <= max_amount:
                            count += 1
                            quantity[symbol] = (max_amount / (quantity_1 * close))
                            quantity1 = quantity[symbol]
                            buy_open.append(open1)
                            high[symbol] = high1
                                      
                            '''
                            client.order_limit_buy(
                                symbol=symbol,
                                quantity=quantity[symbol],
                                price=close  # comment this line to buy at market price
                                )
                            '''
                            spent_amount += close * quantity1
                            buy[symbol] = close
                            print('Bought ' + symbol + ' at ' + str(close))

                            df1 = pd.DataFrame({'Datetime': [datetime.now(tz)], 'Symbol': [symbol], 'Buy/Sell': ['Buy'],
                                        'Quantity': [quantity1], 'Price': [close], 'Profit/loss': [0]})
                            df1['Datetime'] = df1['Datetime'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
                            if not os.path.isfile('Binance.csv'):
                                df1.to_csv('Binance.csv', index=False)
                            else:
                                df1.to_csv('Binance.csv', index=False, mode='a', header=False)

                if symbol in buy:
                    #print(close, high[symbol], high1)
                    if (close <= high[symbol] * (1+sell_percent)) or (close  <= (1+loss_percent) * buy[symbol]):
                        '''
                        client.order_limit_sell(
                          symbol=symbol,
                          quantity=quantity[symbol],
                          price=close  # comment this line to sell at market price
                          )
                        '''
                        if close <= high[symbol]*(1+sell_percent):
                            print('high limit' )
                        profit = close - buy[symbol]
                        max_amount += profit
                        quantity1 = quantity[symbol]
                        spent_amount -= quantity1 * buy[symbol]
                        total_profit = profit * quantity1
                        print("SELL " + symbol + " at " + str(close))
                        print("Profit made " + str(total_profit))

                        df2 = pd.DataFrame({'Datetime': [datetime.now(tz)], 'Symbol': [symbol], 'Buy/Sell': ['Sell'],
                                        'Quantity': [quantity1], 'Price': [close], 'Profit/loss': [total_profit]})
                        df2['Datetime'] = df2['Datetime'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
                        df2.to_csv('Binance.csv', index=False, mode='a', header=False)

                        loss.append(total_profit)
                        if count >= len(list_of_symbols):
                            loss.pop(0)
                        buy.pop(symbol)  # Removing the sold symbol

                if (loss_limit > sum(loss)) or (count > int(transaction)):
                    print("Quitting....")
                    raise SystemExit

            time.sleep(sleep_time)

        except BinanceAPIException as e:
            print(e,symbol)
        except KeyboardInterrupt:
            print("Total Profit " + str(sum(loss)))
            break

        except SystemExit:
            print("Exit")
            print("Total Profit " + str(sum(loss)))
            break


In [13]:
if __name__ == "__main__":
    print("Inside main")
    Main()

Inside main
XMRUSDT
            date   open  close   high    low     volume
0     2020-05-22  62.19  62.22  62.27  62.19  119.19960
1     2020-05-22  62.20  62.12  62.22  62.12  115.91307
2     2020-05-22  62.12  62.20  62.20  62.09  255.29778
3     2020-05-22  62.22  62.23  62.26  62.18  122.97862
4     2020-05-22  62.23  62.36  62.36  62.23  118.53961
...          ...    ...    ...    ...    ...        ...
1435  2020-05-27  63.35  63.47  63.49  63.35   87.75154
1436  2020-05-27  63.49  63.41  63.56  63.40  152.67882
1437  2020-05-27  63.36  63.35  63.39  63.28  497.97340
1438  2020-05-27  63.39  63.42  63.57  63.38  312.02039
1439  2020-05-27  63.44  63.51  63.55  63.44   28.31096

[1440 rows x 6 columns]
LINKUSDT
            date    open   close    high     low    volume
0     2020-05-22  3.9680  3.9618  3.9680  3.9618  11740.11
1     2020-05-22  3.9618  3.9610  3.9621  3.9541  19804.99
2     2020-05-22  3.9619  3.9677  3.9754  3.9619  32496.62
3     2020-05-22  3.9679  3.9735  3.97

In [10]:
import lxml.html as lh
import time
import urllib.request
import argparse
import urllib.request, urllib.parse, urllib.error
import datetime
import pytz
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
import os
import coloredlogs
import json
import ssl
from docopt import docopt
from trading_bot.ops import get_state
from trading_bot.agent import Agent
from trading_bot.methods import evaluate_model
from trading_bot.utils import (
    get_stock_data,
    format_currency,
    format_position,
    show_eval_result,
    switch_k_backend_device
)

Using TensorFlow backend.


In [11]:
tz = pytz.timezone('Asia/Kolkata')
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [12]:
def Real(url):
    uh = urllib.request.urlopen(url, context=ctx)
    data = uh.read().decode()
    info = json.loads(data)
    live = info['price'];
    time.sleep(1)
    return float(live)

In [13]:
def main(args):
    count = 0
    total_profit = 0
    t=0
    history = []
    reward = 0
    price = []
    window_size =5
    time_now = datetime.datetime.now(tz).time()
    url = 'https://api.binance.com/api/v1/ticker/price?symbol={}'.format(args.ticker)
    live = Real(url)
    print(live)

    while(count < 10):
        url = 'https://api.binance.com/api/v1/ticker/price?symbol={}'.format(args.ticker)
        live = Real(url)
        count+=1        
        price.append(live)
        if count < window_size:
           continue
        model_name='model_double-dqn_GOOG_50'  
        print(live)
        initial_offset = price[1] - price[0]
        state = get_state(price, 0, window_size + 1)
        next_state = get_state(price, t + 1, window_size + 1)
        agent = Agent(state_size=window_size, pretrained=True, model_name=model_name)
        agent.inventory = []
        profit = evaluate_model(agent,state,next_state, price, t, total_profit, history, reward, window_size=window_size)
        show_eval_result(model_name, profit, initial_offset)
        t+=1
        state = next_state

In [14]:
def evaluate_model(agent, state, next_state, data, t, total_profit, history, reward, window_size, debug=False):
    print("t")
    print(t)
        # select an action
    action = agent.act(state, is_eval=True)

        # BUY
    if action == 1:
        agent.inventory.append(data[t])

        history.append((data[t], "BUY"))
        if debug:
            logging.debug("Buy at: {}".format(format_currency(data[t])))

        # SELL
    elif action == 2 and len(agent.inventory) > 0:
        bought_price = agent.inventory.pop(0)
        delta = data[t] - bought_price
        reward = delta #max(delta, 0)
        total_profit += delta

        history.append((data[t], "SELL"))
        if debug:
            logging.debug("Sell at: {} | Position: {}".format(
                    format_currency(data[t]), format_position(data[t] - bought_price)))
        # HOLD
    else:
        history.append((data[t], "HOLD"))

#        done = (t == data_length - 1)
    agent.memory.append((state, action, reward, next_state))

    return total_profit


In [25]:

coloredlogs.install(level="DEBUG")
switch_k_backend_device()
args = argparse.Namespace(ticker='BTCUSDT')
try:
    main(args)
except KeyboardInterrupt:
    print("Aborted")    

2020-05-26 11:34:03 darkdevil root[17113] DEBUG switching to TensorFlow for CPU


8928.87
8927.83


2020-05-26 11:34:11 darkdevil root[17113] INFO model_double-dqn_GOOG_50: USELESS



t
0
8928.13


2020-05-26 11:34:13 darkdevil root[17113] INFO model_double-dqn_GOOG_50: USELESS



...

t
3
8927.44


2020-05-26 11:34:19 darkdevil root[17113] INFO model_double-dqn_GOOG_50: USELESS



t
4
8927.27


2020-05-26 11:34:21 darkdevil root[17113] INFO model_double-dqn_GOOG_50: USELESS



t
5
